## Convnet trained from scratch

#### Data pre-processing and data augmentation

" Data augmentation is one way to fight overfitting, but it isn't enough since our augmented samples are still highly correlated. Your main focus for fighting overfitting should be the **entropic capacity** of your model --how much information your model is allowed to store. A model that can store a lot of information has the potential to be more accurate by leveraging more features, but it is also more at risk to start storing irrelevant features. Meanwhile, a model that can only store a few features will have to focus on the most significant features found in the data, and these are more likely to be truly relevant and to generalize better.

There are different ways to modulate entropic capacity. The main one is the choice of the number of parameters in your model, i.e. the number of layers and the size of each layer. Another way is the use of weight regularization, such as $L_1$ or $L_2$ regularization, which consists in forcing model weights to taker smaller values."

"Dropout also helps reduce overfitting, by preventing a layer from seeing twice the exact same pattern, thus acting in a way analoguous to data augmentation (you could say that both dropout and data augmentation tend to disrupt random correlations occuring in your data)."

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from model import create_model
from math import ceil

Using TensorFlow backend.


In [2]:
model = create_model()
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Prepare the data

In [3]:
# using flow_from_directory(), generate batches of image data (and their labels)
batch_size = 40

train_datagen = ImageDataGenerator(
            rotation_range=10, # range (0-180) within which to randomly rotate pictures
            # width_shift_range=0.2, # as fraction of width, range within to which randomly translate pictures
            # height_shift_range=0.2, # same as above, but with height
            rescale=1./255, # RBG coefficient values 0-255 are too hight to process. instead, represent them as values 0-1
            # shear_range=0.2, # random shearing transformations
            zoom_range=0.1, # randomly zooming inside pictures
            horizontal_flip=False,
            fill_mode='nearest') # strategy for filling in newly created pixels, which can appear after a rotation or a width/height shift

val_datagen = ImageDataGenerator(
            rotation_range=10, # range (0-180) within which to randomly rotate pictures
            # width_shift_range=0.2, # as fraction of width, range within to which randomly translate pictures
            # height_shift_range=0.2, # same as above, but with height
            rescale=1./255, # RBG coefficient values 0-255 are too hight to process. instead, represent them as values 0-1
            # shear_range=0.2, # random shearing transformations
            zoom_range=0.1, # randomly zooming inside pictures
            horizontal_flip=False,
            fill_mode='nearest') # strategy for filling in newly created pixels, which can appear after a rotation or a width/height shift

Generate batches of augmented image data, given the path of the original images

In [5]:
# instantiate generators of augmented image batches (and their labels) via .flow(data, labels) or .flow_from_directory(directory)
# this generator will read pictures found in subfolders and indefinitely generate batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '../../../dataset/split_data/train/',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
val_generator = val_datagen.flow_from_directory(
        '../../../dataset/split_data/validation/',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 13447 images belonging to 10 classes.
Found 4487 images belonging to 10 classes.


Use the generators above to train the model.

In [6]:
filepath="weights.h5"

checkpoint_callback = ModelCheckpoint(
                        filepath,
                        monitor='val_acc',
                        verbose=1,
                        save_best_only=True,
                        mode='max')

early_stop_callback = EarlyStopping(
                monitor='val_acc',
                # verbose=0, # decides what to print
                # min_delta=0, # threshold to whether quantify a loss at some epoch as improvement or not. If the difference of loss is below min_delta, it is quantified as no improvement
                # mode='auto', # depends on the direction of the monitored quantity (is it supposed to be decreasing or increasing), since we monitor the loss, we can use min.        
                patience=3,
                mode='max') 

callbacks_list = [checkpoint_callback, early_stop_callback]

In [7]:
history = model.fit_generator(
            train_generator,
            steps_per_epoch=ceil(len(train_generator.filenames) // batch_size),
            epochs=50,
            validation_data=val_generator,
            validation_steps=ceil(len(val_generator.filename) // batch_size),
            callbacks=callbacks_list)

Epoch 1/50
  6/336 [..............................] - ETA: 6:07 - loss: 2.4509 - acc: 0.0792

KeyboardInterrupt: 